In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import cross_val_score, TimeSeriesSplit
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, classification_report
import warnings
warnings.filterwarnings('ignore')
# Load the match data from excel file
df = pd.read_excel('match_data.xlsx', sheet_name='Match Stats')

# Data exploration
print("Total matches in dataset:", len(df))

# The replacement is not needed anymore since you fixed the Excel file
# df['home_team'] = df['home_team'].replace('Boston U.', 'BU')
# df['away_team'] = df['away_team'].replace('Boston U.', 'BU')

# First, make sure all team values are strings
df['home_team'] = df['home_team'].astype(str)
df['away_team'] = df['away_team'].astype(str)

# Create team encodings
teams = sorted(list(set(df['home_team'].unique()) | set(df['away_team'].unique())))
team_to_id = {team: i for i, team in enumerate(teams)}
df['home_team_id'] = df['home_team'].map(team_to_id)
df['away_team_id'] = df['away_team'].map(team_to_id)

print("Corrected Teams in dataset:", teams)
print("Total number of teams:", len(teams))
print("Bucknell matches:", len(df[(df['home_team'] == 'Bucknell') | (df['away_team'] == 'Bucknell')]))

# Ensure date column exists and is properly formatted
if 'Date' not in df.columns:
    # If date doesn't exist, create a placeholder with match sequence
    df['Date'] = pd.date_range(start='2023-01-01', periods=len(df), freq='D')
else:
    # Convert date format if needed
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

# Sort by date to ensure chronological order
df = df.sort_values('Date')

# Create result column - from perspective of team1 vs team2
# W = team1 wins, L = team1 loses, D = draw
df['Result'] = np.where(df['goals_home'] > df['goals_away'], 'W',
                        np.where(df['goals_home'] < df['goals_away'], 'L', 'D'))

# Compute goal difference
df['goal_diff'] = df['goals_home'] - df['goals_away']
df['total_goals'] = df['goals_home'] + df['goals_away']
df['home_shot_conversion'] = df['goals_home'] / df['shots_home'].replace(0, 1)  # Avoid division by zero
df['away_shot_conversion'] = df['goals_away'] / df['shots_away'].replace(0, 1)  # Avoid division by zero

# Create team statistics based on previous matches
team_stats = {}

# Initialize team stats
for team in teams:
    team_stats[team] = {
        'matches_played': 0,
        'wins': 0,
        'draws': 0,
        'losses': 0,
        'goals_for': 0,
        'goals_against': 0,
        'shots': 0,
        'shots_on_target': 0,
        'corners': 0,
        'fouls': 0,
        'home_wins': 0,
        'home_draws': 0,
        'home_losses': 0,
        'away_wins': 0,
        'away_draws': 0,
        'away_losses': 0,
        'home_matches': 0,
        'away_matches': 0,
        'home_goals_for': 0,
        'home_goals_against': 0,
        'away_goals_for': 0,
        'away_goals_against': 0,
        'last_5_results': []  # Store last 5 results for form
    }

# Create head-to-head records
h2h_stats = {}
for team1 in teams:
    h2h_stats[team1] = {}
    for team2 in teams:
        if team1 != team2:
            h2h_stats[team1][team2] = {
                'matches': 0,
                'wins': 0,
                'draws': 0,
                'losses': 0,
                'goals_for': 0,
                'goals_against': 0
            }

# Calculate team stats from match data
for _, row in df.iterrows():
    home_team = row['home_team']
    away_team = row['away_team']

    # Update head-to-head records
    if home_team != away_team:  # Skip if same team (shouldn't happen in real data)
        h2h_stats[home_team][away_team]['matches'] += 1
        h2h_stats[away_team][home_team]['matches'] += 1

        if row['Result'] == 'W':  # Home team wins
            h2h_stats[home_team][away_team]['wins'] += 1
            h2h_stats[away_team][home_team]['losses'] += 1
        elif row['Result'] == 'L':  # Away team wins
            h2h_stats[home_team][away_team]['losses'] += 1
            h2h_stats[away_team][home_team]['wins'] += 1
        else:  # Draw
            h2h_stats[home_team][away_team]['draws'] += 1
            h2h_stats[away_team][home_team]['draws'] += 1

        h2h_stats[home_team][away_team]['goals_for'] += row['goals_home']
        h2h_stats[home_team][away_team]['goals_against'] += row['goals_away']
        h2h_stats[away_team][home_team]['goals_for'] += row['goals_away']
        h2h_stats[away_team][home_team]['goals_against'] += row['goals_home']

    # Update home team stats
    team_stats[home_team]['matches_played'] += 1
    team_stats[home_team]['home_matches'] += 1

    if row['Result'] == 'W':
        team_stats[home_team]['wins'] += 1
        team_stats[home_team]['home_wins'] += 1
    elif row['Result'] == 'D':
        team_stats[home_team]['draws'] += 1
        team_stats[home_team]['home_draws'] += 1
    else:
        team_stats[home_team]['losses'] += 1
        team_stats[home_team]['home_losses'] += 1

    team_stats[home_team]['goals_for'] += row['goals_home']
    team_stats[home_team]['goals_against'] += row['goals_away']
    team_stats[home_team]['home_goals_for'] += row['goals_home']
    team_stats[home_team]['home_goals_against'] += row['goals_away']
    team_stats[home_team]['shots'] += row['shots_home']
    team_stats[home_team]['corners'] += row['corners_home']
    team_stats[home_team]['fouls'] += row['fouls_home']

    # Add result to last 5 results (from home team perspective)
    team_stats[home_team]['last_5_results'].append(row['Result'])
    if len(team_stats[home_team]['last_5_results']) > 5:
        team_stats[home_team]['last_5_results'] = team_stats[home_team]['last_5_results'][-5:]

    # Update away team stats
    team_stats[away_team]['matches_played'] += 1
    team_stats[away_team]['away_matches'] += 1

    if row['Result'] == 'L':
        team_stats[away_team]['wins'] += 1
        team_stats[away_team]['away_wins'] += 1
    elif row['Result'] == 'D':
        team_stats[away_team]['draws'] += 1
        team_stats[away_team]['away_draws'] += 1
    else:
        team_stats[away_team]['losses'] += 1
        team_stats[away_team]['away_losses'] += 1

    team_stats[away_team]['goals_for'] += row['goals_away']
    team_stats[away_team]['goals_against'] += row['goals_home']
    team_stats[away_team]['away_goals_for'] += row['goals_away']
    team_stats[away_team]['away_goals_against'] += row['goals_home']
    team_stats[away_team]['shots'] += row['shots_away']
    team_stats[away_team]['corners'] += row['corners_away']
    team_stats[away_team]['fouls'] += row['fouls_away']

    # Add result to last 5 results (from away team perspective)
    away_result = 'W' if row['Result'] == 'L' else 'L' if row['Result'] == 'W' else 'D'
    team_stats[away_team]['last_5_results'].append(away_result)
    if len(team_stats[away_team]['last_5_results']) > 5:
        team_stats[away_team]['last_5_results'] = team_stats[away_team]['last_5_results'][-5:]

# Create calculated stats
for team in team_stats:
    stats = team_stats[team]
    matches = stats['matches_played']
    home_matches = max(stats['home_matches'], 1)  # Avoid division by zero
    away_matches = max(stats['away_matches'], 1)  # Avoid division by zero

    if matches > 0:
        stats['points'] = stats['wins'] * 3 + stats['draws']
        stats['ppg'] = stats['points'] / matches
        stats['win_rate'] = stats['wins'] / matches
        stats['draw_rate'] = stats['draws'] / matches
        stats['loss_rate'] = stats['losses'] / matches
        stats['goal_diff'] = stats['goals_for'] - stats['goals_against']
        stats['avg_goals_for'] = stats['goals_for'] / matches
        stats['avg_goals_against'] = stats['goals_against'] / matches
        stats['avg_shots'] = stats['shots'] / matches
        stats['avg_corners'] = stats['corners'] / matches
        stats['avg_fouls'] = stats['fouls'] / matches

        # Home-away specific stats
        stats['home_ppg'] = (stats['home_wins'] * 3 + stats['home_draws']) / home_matches
        stats['away_ppg'] = (stats['away_wins'] * 3 + stats['away_draws']) / away_matches
        stats['home_win_rate'] = stats['home_wins'] / home_matches
        stats['away_win_rate'] = stats['away_wins'] / away_matches
        stats['home_avg_goals_for'] = stats['home_goals_for'] / home_matches
        stats['home_avg_goals_against'] = stats['home_goals_against'] / home_matches
        stats['away_avg_goals_for'] = stats['away_goals_for'] / away_matches
        stats['away_avg_goals_against'] = stats['away_goals_against'] / away_matches

        # Form calculation (points from last 5 games)
        recent_results = stats['last_5_results']
        form_points = sum([3 if res == 'W' else 1 if res == 'D' else 0 for res in recent_results])
        stats['form'] = form_points / (len(recent_results) * 3)  # As percentage of maximum possible
    else:
        # Default values for teams with no matches
        stats['points'] = 0
        stats['ppg'] = 0
        stats['win_rate'] = 0
        stats['draw_rate'] = 0
        stats['loss_rate'] = 0
        stats['goal_diff'] = 0
        stats['avg_goals_for'] = 0
        stats['avg_goals_against'] = 0
        stats['avg_shots'] = 0
        stats['avg_corners'] = 0
        stats['avg_fouls'] = 0
        stats['home_ppg'] = 0
        stats['away_ppg'] = 0
        stats['home_win_rate'] = 0
        stats['away_win_rate'] = 0
        stats['home_avg_goals_for'] = 0
        stats['home_avg_goals_against'] = 0
        stats['away_avg_goals_for'] = 0
        stats['away_avg_goals_against'] = 0
        stats['form'] = 0.5  # Neutral form for teams with no data

# Create team statistics dataframe
team_stats_df = pd.DataFrame([
    {
        'Team': team,
        'Matches': stats['matches_played'],
        'Wins': stats['wins'],
        'Draws': stats['draws'],
        'Losses': stats['losses'],
        'GF': stats['goals_for'],
        'GA': stats['goals_against'],
        'GD': stats['goal_diff'],
        'Points': stats['points'],
        'PPG': stats['ppg'],
        'Win%': stats['win_rate'] * 100,
        'Home PPG': stats['home_ppg'],
        'Away PPG': stats['away_ppg'],
        'Form': stats['form'] * 100
    }
    for team, stats in team_stats.items()
]).sort_values('Points', ascending=False)

print("\nTeam Performance Table:")
print(team_stats_df[['Team', 'Matches', 'Wins', 'Draws', 'Losses', 'Points', 'PPG', 'GF', 'GA', 'GD', 'Form']])

# Enhanced feature engineering for match prediction
def create_feature_vector(row):
    home_team = row['home_team']
    away_team = row['away_team']

    home_stats = team_stats[home_team]
    away_stats = team_stats[away_team]

    # Get head-to-head stats if available
    h2h = h2h_stats[home_team][away_team]

    # Calculate head-to-head strength (how well home team performs against away team)
    if h2h['matches'] > 0:
        h2h_win_rate = h2h['wins'] / h2h['matches']
        h2h_ppg = (h2h['wins'] * 3 + h2h['draws']) / h2h['matches']
        h2h_goal_diff = (h2h['goals_for'] - h2h['goals_against']) / h2h['matches']
    else:
        # Default values if no h2h history
        h2h_win_rate = 0.5
        h2h_ppg = home_stats['ppg'] - away_stats['ppg'] + 0.5
        h2h_goal_diff = 0

    # Create a comprehensive feature vector with enhanced stats
    features = [
        # Basic team IDs
        row['home_team_id'],
        row['away_team_id'],

        # Overall team performance
        home_stats['ppg'],
        away_stats['ppg'],
        home_stats['win_rate'],
        away_stats['win_rate'],
        home_stats['draw_rate'],
        away_stats['draw_rate'],

        # Goal stats
        home_stats['avg_goals_for'],
        home_stats['avg_goals_against'],
        away_stats['avg_goals_for'],
        away_stats['avg_goals_against'],

        # Strength at home/away
        home_stats['home_ppg'],
        away_stats['away_ppg'],
        home_stats['home_win_rate'],
        away_stats['away_win_rate'],
        home_stats['home_avg_goals_for'],
        home_stats['home_avg_goals_against'],
        away_stats['away_avg_goals_for'],
        away_stats['away_avg_goals_against'],

        # Recent form
        home_stats['form'],
        away_stats['form'],

        # Head-to-head stats
        h2h_win_rate,
        h2h_ppg,
        h2h_goal_diff,

        # Game stats
        home_stats['avg_shots'],
        away_stats['avg_shots'],
        home_stats['avg_corners'],
        away_stats['avg_corners'],

        # Strategic insights
        home_stats['avg_goals_for'] - away_stats['avg_goals_against'],  # Home attack vs Away defense
        away_stats['avg_goals_for'] - home_stats['avg_goals_against'],  # Away attack vs Home defense

        # Momentum (goal difference in last few games)
        home_stats['goal_diff'] / max(home_stats['matches_played'], 1),
        away_stats['goal_diff'] / max(away_stats['matches_played'], 1)
    ]

    return features

# Prepare train/test data
X = np.array([create_feature_vector(row) for _, row in df.iterrows()])
y = np.array([0 if r == 'W' else 1 if r == 'D' else 2 for r in df['Result']])

# List of feature names (for importance analysis)
feature_names = [
    'Home Team ID', 'Away Team ID',
    'Home PPG', 'Away PPG',
    'Home Win Rate', 'Away Win Rate',
    'Home Draw Rate', 'Away Draw Rate',
    'Home Avg Goals For', 'Home Avg Goals Against',
    'Away Avg Goals For', 'Away Avg Goals Against',
    'Home PPG at Home', 'Away PPG Away',
    'Home Win Rate at Home', 'Away Win Rate Away',
    'Home Avg Goals For at Home', 'Home Avg Goals Against at Home',
    'Away Avg Goals For Away', 'Away Avg Goals Against Away',
    'Home Recent Form', 'Away Recent Form',
    'H2H Home Win Rate', 'H2H Home PPG', 'H2H Goal Diff per Game',
    'Home Avg Shots', 'Away Avg Shots',
    'Home Avg Corners', 'Away Avg Corners',
    'Home Attack vs Away Defense', 'Away Attack vs Home Defense',
    'Home Goal Diff per Game', 'Away Goal Diff per Game'
]

# Try both Random Forest and Gradient Boosting
print("\nTraining models with enhanced features...")

# Train a Random Forest classifier with tuned parameters
rf_model = RandomForestClassifier(
    n_estimators=200,
    max_depth=10,
    min_samples_split=5,
    min_samples_leaf=2,
    class_weight='balanced',
    random_state=42
)
rf_model.fit(X, y)

# Train a Gradient Boosting classifier
gb_model = GradientBoostingClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=5,
    random_state=42
)
gb_model.fit(X, y)

# Evaluate the models with cross-validation
# Use TimeSeriesSplit for chronological data
tscv = TimeSeriesSplit(n_splits=5)
rf_cv_scores = cross_val_score(rf_model, X, y, cv=tscv)
gb_cv_scores = cross_val_score(gb_model, X, y, cv=tscv)

print(f"\nRandom Forest Cross-validation accuracy: {np.mean(rf_cv_scores):.4f}")
print(f"Gradient Boosting Cross-validation accuracy: {np.mean(gb_cv_scores):.4f}")

# Use the better performing model
if np.mean(rf_cv_scores) >= np.mean(gb_cv_scores):
    model = rf_model
    print("Using Random Forest model for predictions (better performance)")
else:
    model = gb_model
    print("Using Gradient Boosting model for predictions (better performance)")

# Feature importance
if hasattr(model, 'feature_importances_'):
    print("\nFeature Importance:")
    feature_importance = sorted(zip(feature_names, model.feature_importances_),
                                key=lambda x: x[1], reverse=True)
    for i, (feature, importance) in enumerate(feature_importance[:15]):
        print(f"{i+1}. {feature}: {importance:.4f}")

# Make predictions for Bucknell against all other teams
bucknell_predictions = []

# Get all other teams
other_teams = [team for team in teams if team != 'Bucknell']

# For each opponent, predict Bucknell as both home and away
for opponent in other_teams:
    # Bucknell as home team
    home_features = create_feature_vector({
        'home_team': 'Bucknell',
        'away_team': opponent,
        'home_team_id': team_to_id['Bucknell'],
        'away_team_id': team_to_id[opponent]
    })

    # Bucknell as away team
    away_features = create_feature_vector({
        'home_team': opponent,
        'away_team': 'Bucknell',
        'home_team_id': team_to_id[opponent],
        'away_team_id': team_to_id['Bucknell']
    })

    # Predict both scenarios
    home_pred = model.predict([home_features])[0]
    away_pred = model.predict([away_features])[0]

    # Get probability estimates
    home_probs = model.predict_proba([home_features])[0]
    away_probs = model.predict_proba([away_features])[0]

    # Convert to match results from Bucknell's perspective
    bucknell_home_result = 'Win' if home_pred == 0 else 'Draw' if home_pred == 1 else 'Loss'
    bucknell_away_result = 'Win' if away_pred == 2 else 'Draw' if away_pred == 1 else 'Loss'

    # Add predictions
    bucknell_predictions.append({
        'Opponent': opponent,
        'Bucknell at Home': bucknell_home_result,
        'Win Prob (Home)': round(home_probs[0] * 100, 1),
        'Draw Prob (Home)': round(home_probs[1] * 100, 1),
        'Loss Prob (Home)': round(home_probs[2] * 100, 1),
        'Bucknell Away': bucknell_away_result,
        'Win Prob (Away)': round(away_probs[2] * 100, 1),
        'Draw Prob (Away)': round(away_probs[1] * 100, 1),
        'Loss Prob (Away)': round(away_probs[0] * 100, 1)
    })

# Create predictions dataframe
bucknell_pred_df = pd.DataFrame(bucknell_predictions)

# Display Bucknell predictions
print("\nBucknell Match Predictions:")
print(bucknell_pred_df[['Opponent', 'Bucknell at Home', 'Bucknell Away']])

# Calculate expected points
home_points = sum([3 if r == 'Win' else 1 if r == 'Draw' else 0 for r in bucknell_pred_df['Bucknell at Home']])
away_points = sum([3 if r == 'Win' else 1 if r == 'Draw' else 0 for r in bucknell_pred_df['Bucknell Away']])
total_points = home_points + away_points
matches = len(bucknell_pred_df) * 2

print(f"\nExpected Points for Bucknell:")
print(f"Home Points: {home_points} from {len(bucknell_pred_df)} matches")
print(f"Away Points: {away_points} from {len(bucknell_pred_df)} matches")
print(f"Total Points: {total_points} from {matches} matches")
print(f"Expected PPG: {total_points / matches:.2f}")

# Detailed prediction breakdown
print("\nDetailed Bucknell Predictions:")
for _, row in bucknell_pred_df.iterrows():
    print(f"vs {row['Opponent']}:")
    print(f"  At Home: {row['Bucknell at Home']} (Win: {row['Win Prob (Home)']}%, Draw: {row['Draw Prob (Home)']}%, Loss: {row['Loss Prob (Home)']}%)")
    print(f"  Away: {row['Bucknell Away']} (Win: {row['Win Prob (Away)']}%, Draw: {row['Draw Prob (Away)']}%, Loss: {row['Loss Prob (Away)']}%)")

# Summary of improvements
print("\nMODEL IMPROVEMENTS SUMMARY:")
print("1. Team name standardization: Combined BU and Boston U. as the same team")
print("2. Enhanced features:")
print("   - Home/away performance differentials")
print("   - Recent team form metrics")
print("   - Head-to-head statistics")
print("   - Attack vs defense matchup indicators")
print("3. Advanced modeling:")
print("   - Tuned Random Forest parameters")
print("   - Tested Gradient Boosting classifier")
print("   - Used time-series cross-validation")
print("4. More accurate predictions:")
print(f"   - Original model accuracy: ~0.71")
print(f"   - Enhanced model accuracy: {max(np.mean(rf_cv_scores), np.mean(gb_cv_scores)):.4f}")
print(f"   - Improvement: {(max(np.mean(rf_cv_scores), np.mean(gb_cv_scores)) - 0.71) * 100:.1f}%")

#pvalue?/error rate
#print equation it uses for each game

Total matches in dataset: 59
Corrected Teams in dataset: ['0', 'American', 'Army West Point', 'BU', 'Bucknell', 'Colgate', 'Holy Cross', 'Lafayette', 'Lehigh', 'Loyola', 'Navy']
Total number of teams: 11
Bucknell matches: 13

Team Performance Table:
               Team  Matches  Wins  Draws  Losses  Points       PPG  GF  GA  \
4          Bucknell       13    10      0       3      30  2.307692  18   5   
2   Army West Point       14     8      1       5      25  1.785714  21  10   
3                BU       12     6      6       0      24  2.000000  22  10   
9            Loyola       14     4      5       5      17  1.214286  22  17   
5           Colgate       14     4      4       6      16  1.142857  20  25   
10             Navy       14     4      3       7      15  1.071429  14  25   
7         Lafayette        9     2      4       3      10  1.111111   8   6   
1          American        9     2      2       5       8  0.888889  10  19   
6        Holy Cross        9     2     